In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import csv

#### Ustanawiam obiekt webdrivera, w celu rozpoczęcia pracy

In [2]:
browser = webdriver.Firefox(executable_path=r'D:\geckodriver-v0.29.0-win64\geckodriver.exe')
browser.get('https://www.cgen.pl/indeksy/ocena/samice/ranking')

#### BeautifulSoup jak parser języka html

In [3]:
html = browser.page_source
soup = BeautifulSoup(html, "html.parser")

In [4]:
title = soup.title #nazwa strony
print(title)

<title> Ocena wartości hodowlanej</title>


### 1. Dane jałówek

#### Szukam rozwijanego przycisku, do wyboru ilości wpisów na stronie

In [5]:
przyc=soup.find_all("button")
for x in przyc:
    print(x)

<button class="btn btn-default" style="background-color: rgb(231, 231, 231);" type="button"> <span class="flag-icon flag-icon-gb"></span></button>
<button aria-expanded="false" aria-haspopup="true" class="btn btn-default btn-secondary dropdown-toggle" data-toggle="dropdown" id="pageDropDown">20<span> <span class="caret"></span></span></button>


In [6]:
rozwijacz=przyc[1]
print(rozwijacz)

<button aria-expanded="false" aria-haspopup="true" class="btn btn-default btn-secondary dropdown-toggle" data-toggle="dropdown" id="pageDropDown">20<span> <span class="caret"></span></span></button>


#### Szukam elementu o powyższych parametrach

In [7]:
rozwijac_kafelek=browser.find_element_by_id('pageDropDown')
rozwijac_kafelek.click()

#### Pozycje przycisku do wyboru ilosci wpisow na stronie - interesuje nas 500

In [8]:
pozycje_przycisku=soup.find_all('li',{'class': "dropdown-item"})
for x in pozycje_przycisku:
    print(x.find("a"))

<a data-page="20" href="#" role="menuitem" tabindex="-1">20</a>
<a data-page="50" href="#" role="menuitem" tabindex="-1">50</a>
<a data-page="100" href="#" role="menuitem" tabindex="-1">100</a>
<a data-page="500" href="#" role="menuitem" tabindex="-1">500</a>


In [9]:
rozwijacz= pozycje_przycisku[3].find("a")
print(rozwijacz)

<a data-page="500" href="#" role="menuitem" tabindex="-1">500</a>


#### Ponownie szukam elementu o powyższych parametrach

In [10]:
pozycja_500=browser.find_element_by_xpath("//a[@data-page='500']")
pozycja_500.click()

#### Ponownie pobieram dane ze strony - aktualizuje obiekt webdrivera

In [11]:
html = browser.page_source
soup = BeautifulSoup(html, "html.parser")

#### Lokalizuje obiekt w którym znadują się intersujące mnie dane

In [12]:
tabele=soup.find_all("table")

In [13]:
print(len(tabele))
for x in tabele:
    print(x,"\n")
#1 tabela to nagłówki, 2 to treść właściwa

2
<table class="table table-hover table-bordered table-condensed"><colgroup><col style="width: 80px; min-width: 80px;"/><col style="width: 125px; min-width: 125px;"/><col style="width: 125px; min-width: 125px;"/><col style="width: 125px; min-width: 125px;"/><col style="width: 100px; min-width: 100px;"/><col style="width: 165px; min-width: 165px;"/><col style="width: 150px; min-width: 150px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-width: 70px;"/><col style="width: 70px; min-wid

#### Pobieram pierwsze nagłówek tabeli i pierwsze 500 wpisów w postaci data-frama

In [14]:
nagłówek=pd.read_html(str(tabele[0]),header=0)[0]

In [15]:
nagłówek

,Pozycja,ID,Nazwa ojca,ID ojca,Obora,Hodowca,Województwo,Data urodzenia,IE,Dokł. IE,...,kg ml,kg tł,% tł,kg bi,% bi,rc,sm,nr,w,og


In [16]:
część_1_1=pd.read_html(str(tabele[1]),header=None)[0]
część_1_1.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,PL005435512906,EXPANSION,CA12857929,514481.0,PIETRUSZYŃSKI ADAM,PODLASKIE,2020,3165,66,...,1477,598,-1,439,-4,106,106,105,118,112
1,2,PL005435514047,EXPANSION,CA12857929,514481.0,PIETRUSZYŃSKI ADAM,PODLASKIE,2020,3010,66,...,1360,567,1,414,-3,104,103,103,116,110
2,3,PL005433807509,GARETT,DE1504336100,902181.0,HZINR POLANOWICE SP. Z O.O.,KUJAWSKO-POMORSKIE,2021,2981,65,...,1083,358,-10,458,12,105,106,125,132,123
3,4,PL005432792516,PREDATOR,DE0540259057,2101481.0,OHZ LUBIANA SP. Z O.O.,ZACHODNIOPOMORSKIE,2020,2941,65,...,885,494,16,379,11,112,110,116,129,125
4,5,PL005456057172,EXPANSION,CA12857929,5300761.0,KR KIETRZ SP. Z O.O.,OPOLSKIE,2020,2939,66,...,1294,477,-6,502,9,106,103,112,122,113


#### Lokalizuje przycisk przejścia do 2 części tabeli 

In [17]:
pozycje_przewijania=soup.find_all('li',{'class': "page-item"})
for x in pozycje_przewijania:
    print(x.find("a"))

<a class="page-link" href="#">1</a>
<a class="page-link" href="#">2</a>
<a class="page-link" href="#">&gt;</a>


In [18]:
p_przewijania= pozycje_przewijania[1].find("a")
print(p_przewijania)

<a class="page-link" href="#">2</a>


#### Z racji braku konkretnego znacznika, wartość  ścieżki xpath wybieram analogicznie do kolejoności instancji

In [19]:
przyciski=browser.find_elements_by_xpath("//a[@class='page-link']")

In [20]:
pozycje=["1","2",">"]
for x in range(0,len(przyciski)):
    print(pozycje[x],"\n",przyciski[x])

1 
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="9525d1ad-067c-491f-b07f-8be05b463a19", element="11e98a36-a787-4506-9911-98687864c00b")>
2 
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="9525d1ad-067c-491f-b07f-8be05b463a19", element="c4b8db96-c2bf-48a1-8e1d-5660eeabde96")>
> 
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="9525d1ad-067c-491f-b07f-8be05b463a19", element="d815b671-7a6a-4e1e-9efa-b918f2451631")>


#### Wywołuję kolejne 500 wpisów

In [21]:
przyciski[1].click()

#### Ponownie aktualizuje obiket Webdrivera

In [22]:
html = browser.page_source
soup = BeautifulSoup(html, "html.parser")

In [23]:
tabele_2=soup.find_all("table")

In [24]:
część_1_2= pd.read_html(str(tabele_2[1]),header=None)[0]
część_1_2.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,501,PL005443260103,FIREBIRD,US3142352969,8701081.0,SK NOWE JANKOWICE SP. Z O.O.,KUJAWSKO-POMORSKIE,2020,2222,66,...,908,418,5,361,8,109,106,104,120,118
1,502,PL005431510395,HUSKY,US3130915944,2101441.0,OHZ LUBIANA SP. Z O.O.,ZACHODNIOPOMORSKIE,2020,2222,65,...,1078,532,11,270,-9,106,103,111,133,122
2,503,PL005487307192,GARETT,DE1504336100,5311071.0,MLEKOLAND SP.Z O.O.,OPOLSKIE,2020,2222,65,...,992,347,-7,276,-5,106,107,116,122,115
3,504,PL005517735124,BIGGIE,US3136807571,NaN,NaN,NaN,2020,2221,66,...,1049,577,17,445,12,105,103,118,114,113
4,505,PL005438537302,CASINO,NL865720444,NaN,NaN,NaN,2020,2221,66,...,722,432,16,273,5,107,104,111,123,115


# Pobieram dane Krów

#### Ponownie aktualizuje obiket Webdrivera

In [25]:
html = browser.page_source
soup = BeautifulSoup(html, "html.parser")

#### Szukam rozwijanych przyciskow IE \ jałówka \ HO \ Genomowa

In [26]:
q=soup.find_all("select", { "id" : "formSelectIndex" })
for x in q:
    print(q,"\n")

[<select class="form-control" id="formSelectIndex"><option style="font-weight: bold;" value="ie"><span>IE</span></option><option style="font-weight: bold;" value="pf_index"><span>PF</span></option></select>, <select class="form-control" id="formSelectIndex"><option value="FemaleType=Heifer"><span>Jałówka</span></option><option value="FemaleType=Cow"><span>Krowa</span></option></select>, <select class="form-control" id="formSelectIndex"><option value="Breed=HO">HO</option><option value="Breed=RW">RW</option></select>, <select class="form-control" id="formSelectIndex"><option value="EvalType[]=G&amp;EvalType[]=Z"><span>Genomowa</span></option></select>] 

[<select class="form-control" id="formSelectIndex"><option style="font-weight: bold;" value="ie"><span>IE</span></option><option style="font-weight: bold;" value="pf_index"><span>PF</span></option></select>, <select class="form-control" id="formSelectIndex"><option value="FemaleType=Heifer"><span>Jałówka</span></option><option value="Fe

In [27]:
przyciski=browser.find_elements_by_xpath("//select[@class='form-control']")
for x in przyciski:
    print(x,"\n")

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="9525d1ad-067c-491f-b07f-8be05b463a19", element="23c41120-83fe-4808-8932-9964f2a6c769")> 

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="9525d1ad-067c-491f-b07f-8be05b463a19", element="d1ee6325-2181-4b26-ad3f-97dfb1a33a6a")> 

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="9525d1ad-067c-491f-b07f-8be05b463a19", element="cbbee50d-c130-4300-9c6f-8c150a000248")> 

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="9525d1ad-067c-491f-b07f-8be05b463a19", element="c995846d-d128-4c3d-8586-c2014cd131ff")> 



#### Klikam rozwijany kafelek jałówka 

In [28]:
przyciski[1].click()

In [29]:
przyciski=browser.find_elements_by_xpath("//option[@value='FemaleType=Cow']")
for x in przyciski:
    print(x,"\n")

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="9525d1ad-067c-491f-b07f-8be05b463a19", element="e65b572a-b237-448b-9431-e86ed9a64f0f")> 



In [30]:
przyciski[0].click()

#### Ponownie aktualizuje obiket Webdrivera

In [31]:
html = browser.page_source
soup = BeautifulSoup(html, "html.parser")

#### Rozwijam kafelkę z liczbami wpisów na stronę

In [32]:
rozwijac_kafelek=browser.find_element_by_id('pageDropDown')
rozwijac_kafelek.click()

In [33]:
pozycje_przycisku=soup.find_all('li',{'class': "dropdown-item"})
for x in pozycje_przycisku:
    print(x.find("a"))

<a data-page="20" href="#" role="menuitem" tabindex="-1">20</a>
<a data-page="50" href="#" role="menuitem" tabindex="-1">50</a>
<a data-page="100" href="#" role="menuitem" tabindex="-1">100</a>
<a data-page="500" href="#" role="menuitem" tabindex="-1">500</a>


In [34]:
rozwijacz= pozycje_przycisku[3].find("a")
print(rozwijacz)

<a data-page="500" href="#" role="menuitem" tabindex="-1">500</a>


In [35]:
pozycja_500=browser.find_element_by_xpath("//a[@data-page='500']")
pozycja_500.click()

#### Ponownie aktualizuje obiket Webdrivera

In [36]:
html = browser.page_source
soup = BeautifulSoup(html, "html.parser")

#### Lokalizuje tabele z danymi

In [37]:
tabele=soup.find_all("table")

In [38]:
część_2_1=pd.read_html(str(tabele[1]),header=None)[0]
część_2_1.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,PL005482473694,BRAVENESS,US3014562366,4106971.0,DANKO HODOWLA ROŚLIN,WIELKOPOLSKIE,2019,2575,66,...,1027,523,12,429,11,110.0,111.0,117.0,128.0,126.0
1,2,PL005371677462,CYRANO,DE0666847135,4106981.0,DANKO HODOWLA ROŚLIN,WIELKOPOLSKIE,2018,2534,69,...,1036,714,34,413,9,111.0,111.0,108.0,122.0,118.0
2,3,PL005337453444,CHARMING,DE0666732945,902181.0,HZINR POLANOWICE SP. Z O.O.,KUJAWSKO-POMORSKIE,2018,2498,68,...,1472,554,-5,432,-5,106.0,101.0,108.0,119.0,108.0
3,4,PL005361717246,FRAZZLED,US74261651,6300261.0,PR-H GAŁOPOL SP. Z O.O.,WIELKOPOLSKIE,2019,2470,67,...,972,530,16,352,4,103.0,105.0,102.0,116.0,107.0
4,5,PL005321922024,FRAZZLED,US74261651,5723781.0,SK DOBRZYNIEWO SP. Z O.O.,KUJAWSKO-POMORSKIE,2018,2467,68,...,1905,463,-33,498,-13,105.0,111.0,106.0,119.0,118.0


#### Przechodze do kolejnych 500 wpisów

In [39]:
przyciski=browser.find_elements_by_xpath("//a[@class='page-link']")

In [40]:
przyciski[1].click()

#### Ponownie aktualizuje obiket Webdrivera

In [41]:
html = browser.page_source
soup = BeautifulSoup(html, "html.parser")

#### Lokalizuje tabele z danymi

In [42]:
tabele=soup.find_all("table")

In [43]:
część_2_2= pd.read_html(str(tabele_2[1]),header=None)[0]
część_2_2.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,501,PL005443260103,FIREBIRD,US3142352969,8701081.0,SK NOWE JANKOWICE SP. Z O.O.,KUJAWSKO-POMORSKIE,2020,2222,66,...,908,418,5,361,8,109,106,104,120,118
1,502,PL005431510395,HUSKY,US3130915944,2101441.0,OHZ LUBIANA SP. Z O.O.,ZACHODNIOPOMORSKIE,2020,2222,65,...,1078,532,11,270,-9,106,103,111,133,122
2,503,PL005487307192,GARETT,DE1504336100,5311071.0,MLEKOLAND SP.Z O.O.,OPOLSKIE,2020,2222,65,...,992,347,-7,276,-5,106,107,116,122,115
3,504,PL005517735124,BIGGIE,US3136807571,NaN,NaN,NaN,2020,2221,66,...,1049,577,17,445,12,105,103,118,114,113
4,505,PL005438537302,CASINO,NL865720444,NaN,NaN,NaN,2020,2221,66,...,722,432,16,273,5,107,104,111,123,115


#### Zmiana nazw kolumn i połączenie danych w 1 data frame

In [44]:
dane_jalowka_i_krowa = pd.concat([część_1_1,część_1_2,część_2_1,część_2_2])
dane_jalowka = pd.concat([część_1_1,część_1_2])

In [45]:
dane_jalowka_i_krowa.reset_index(drop=True)
dane_jalowka.reset_index(drop=True)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,PL005435512906,EXPANSION,CA12857929,514481.0,PIETRUSZYŃSKI ADAM,PODLASKIE,2020,3165,66,...,1477,598,-1,439,-4,106,106,105,118,112
1,2,PL005435514047,EXPANSION,CA12857929,514481.0,PIETRUSZYŃSKI ADAM,PODLASKIE,2020,3010,66,...,1360,567,1,414,-3,104,103,103,116,110
2,3,PL005433807509,GARETT,DE1504336100,902181.0,HZINR POLANOWICE SP. Z O.O.,KUJAWSKO-POMORSKIE,2021,2981,65,...,1083,358,-10,458,12,105,106,125,132,123
3,4,PL005432792516,PREDATOR,DE0540259057,2101481.0,OHZ LUBIANA SP. Z O.O.,ZACHODNIOPOMORSKIE,2020,2941,65,...,885,494,16,379,11,112,110,116,129,125
4,5,PL005456057172,EXPANSION,CA12857929,5300761.0,KR KIETRZ SP. Z O.O.,OPOLSKIE,2020,2939,66,...,1294,477,-6,502,9,106,103,112,122,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,PL005519311449,MEZZO,DE0360914779,545561.0,ZARĘBA RAFAŁ,PODLASKIE,2020,2082,65,...,1149,378,-11,338,-4,115,109,118,123,122
996,997,PL005462558175,OBJECTIVE,FR4407861720,4138011.0,GR MARCIN GRONOWSKI,WIELKOPOLSKIE,2021,2082,65,...,597,395,18,253,7,102,100,113,114,107
997,998,PL005485752857,BIGGIE,US3136807571,5300761.0,KR KIETRZ SP. Z O.O.,OPOLSKIE,2020,2081,67,...,949,317,-8,289,-2,114,111,120,119,121
998,999,PL005383706099,KODEN,DE0359172932,902181.0,HZINR POLANOWICE SP. Z O.O.,KUJAWSKO-POMORSKIE,2019,2081,66,...,741,501,24,299,7,104,106,94,116,111


In [46]:
a=nagłówek.columns
nazwy_kolumn=[]
for x in a:
    nazwy_kolumn.append(x)
dane_jalowka_i_krowa.columns=nazwy_kolumn
dane_jalowka.columns=nazwy_kolumn

In [47]:
dane_jalowka_i_krowa

,Pozycja,ID,Nazwa ojca,ID ojca,Obora,Hodowca,Województwo,Data urodzenia,IE,Dokł. IE,...,kg ml,kg tł,% tł,kg bi,% bi,rc,sm,nr,w,og
0,1,PL005435512906,EXPANSION,CA12857929,514481.0,PIETRUSZYŃSKI ADAM,PODLASKIE,2020,3165,66,...,1477,598,-1,439,-4,106.0,106.0,105.0,118.0,112.0
1,2,PL005435514047,EXPANSION,CA12857929,514481.0,PIETRUSZYŃSKI ADAM,PODLASKIE,2020,3010,66,...,1360,567,1,414,-3,104.0,103.0,103.0,116.0,110.0
2,3,PL005433807509,GARETT,DE1504336100,902181.0,HZINR POLANOWICE SP. Z O.O.,KUJAWSKO-POMORSKIE,2021,2981,65,...,1083,358,-10,458,12,105.0,106.0,125.0,132.0,123.0
3,4,PL005432792516,PREDATOR,DE0540259057,2101481.0,OHZ LUBIANA SP. Z O.O.,ZACHODNIOPOMORSKIE,2020,2941,65,...,885,494,16,379,11,112.0,110.0,116.0,129.0,125.0
4,5,PL005456057172,EXPANSION,CA12857929,5300761.0,KR KIETRZ SP. Z O.O.,OPOLSKIE,2020,2939,66,...,1294,477,-6,502,9,106.0,103.0,112.0,122.0,113.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,996,PL005519311449,MEZZO,DE0360914779,545561.0,ZARĘBA RAFAŁ,PODLASKIE,2020,2082,65,...,1149,378,-11,338,-4,115.0,109.0,118.0,123.0,122.0
496,997,PL005462558175,OBJECTIVE,FR4407861720,4138011.0,GR MARCIN GRONOWSKI,WIELKOPOLSKIE,2021,2082,65,...,597,395,18,253,7,102.0,100.0,113.0,114.0,107.0
497,998,PL005485752857,BIGGIE,US3136807571,5300761.0,KR KIETRZ SP. Z O.O.,OPOLSKIE,2020,2081,67,...,949,317,-8,289,-2,114.0,111.0,120.0,119.0,121.0
498,999,PL005383706099,KODEN,DE0359172932,902181.0,HZINR POLANOWICE SP. Z O.O.,KUJAWSKO-POMORSKIE,2019,2081,66,...,741,501,24,299,7,104.0,106.0,94.0,116.0,111.0


In [48]:
dane_jalowka

,Pozycja,ID,Nazwa ojca,ID ojca,Obora,Hodowca,Województwo,Data urodzenia,IE,Dokł. IE,...,kg ml,kg tł,% tł,kg bi,% bi,rc,sm,nr,w,og
0,1,PL005435512906,EXPANSION,CA12857929,514481.0,PIETRUSZYŃSKI ADAM,PODLASKIE,2020,3165,66,...,1477,598,-1,439,-4,106,106,105,118,112
1,2,PL005435514047,EXPANSION,CA12857929,514481.0,PIETRUSZYŃSKI ADAM,PODLASKIE,2020,3010,66,...,1360,567,1,414,-3,104,103,103,116,110
2,3,PL005433807509,GARETT,DE1504336100,902181.0,HZINR POLANOWICE SP. Z O.O.,KUJAWSKO-POMORSKIE,2021,2981,65,...,1083,358,-10,458,12,105,106,125,132,123
3,4,PL005432792516,PREDATOR,DE0540259057,2101481.0,OHZ LUBIANA SP. Z O.O.,ZACHODNIOPOMORSKIE,2020,2941,65,...,885,494,16,379,11,112,110,116,129,125
4,5,PL005456057172,EXPANSION,CA12857929,5300761.0,KR KIETRZ SP. Z O.O.,OPOLSKIE,2020,2939,66,...,1294,477,-6,502,9,106,103,112,122,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,996,PL005519311449,MEZZO,DE0360914779,545561.0,ZARĘBA RAFAŁ,PODLASKIE,2020,2082,65,...,1149,378,-11,338,-4,115,109,118,123,122
496,997,PL005462558175,OBJECTIVE,FR4407861720,4138011.0,GR MARCIN GRONOWSKI,WIELKOPOLSKIE,2021,2082,65,...,597,395,18,253,7,102,100,113,114,107
497,998,PL005485752857,BIGGIE,US3136807571,5300761.0,KR KIETRZ SP. Z O.O.,OPOLSKIE,2020,2081,67,...,949,317,-8,289,-2,114,111,120,119,121
498,999,PL005383706099,KODEN,DE0359172932,902181.0,HZINR POLANOWICE SP. Z O.O.,KUJAWSKO-POMORSKIE,2019,2081,66,...,741,501,24,299,7,104,106,94,116,111


#### Zapisuje dane obu zbiorów

In [50]:
dane_jalowka_i_krowa.to_csv('Dane_jalowka_i_krowa.csv',index=False, encoding="utf-8")
dane_jalowka.to_csv('Dane_jalowka.csv',index=False, encoding="utf-8")